In [3]:
from secret_key import openai_key
import os
os.environ['OPENAI_API_KEY'] = openai_key

from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.6) #how creative the llm is
'''
name = llm("I want to open an Indian restaurant, suggest me a name for that.")
print(name)
'''

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


prompt_template_name = PromptTemplate(
  input_variables = ['cuisine'],
  template = "I want to open an {cuisine} restaurant, suggest me a name for that" 
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

#prompt_template_name.format(cuisine="Italian")

#name_chain.run("Mexican")

prompt_template_items = PromptTemplate(
  input_variables = ['restaurant_name'],
  template = "Suggest some menu items for {restaurant_name}. Return it as a comma seperatead list." 
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

'''
#for one output
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
print(response)
'''

#multiple outputs
from langchain.chains import SequentialChain

chain = SequentialChain(
  chains = [name_chain, food_items_chain],
  input_variables = ['cuisine'],
  output_variables = ['restaurant_name' , 'menu_items']
)

#chain({'cuisine' : 'Arabic'})

C:\Users\Tuf\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia","llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOR_REACT_DESCRIPTION, #thought the action where to go
    #verbose=True #we can inspect the internal actions
)


agent.run("When was Elon musk born? what is his age right now in 2024?")
    

ImportError: Could not import wikipedia python package. Please install it with `pip install wikipedia`.

In [9]:
from secret_key import serpapi_key
import os

os.environ['SERPAPI_API_KEY'] = serpapi_key

In [10]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

#we're giving the access to tools
tools = load_tools(["serpapi","llm-math"], llm=llm)

#initializing an agent with tools
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("What was the GDP of US in 2023 plus 5?")


ImportError: Could not import serpapi python package. Please install it with `pip install google-search-results`.

## Memory

In [14]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



1. "Casa de Sabor" (House of Flavor)
2. "El Sabroso" (The Tasty One)
3. "La Cocina Mexicana" (The Mexican Kitchen)
4. "Taco Fiesta"
5. "Salsa y Sabor" (Salsa and Flavor)
6. "El Ranchito" (The Little Ranch)
7. "Cantina del Sol" (Cantina of the Sun)
8. "La Taqueria" (The Taco Shop)
9. "El Comal" (The Griddle)
10. "Fiesta Mexicana" (Mexican Party)


In [15]:
type(chain.memory)

NoneType

In [16]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Indian")
print(name)



1. "Spice Junction"
2. "Taste of India"
3. "Curry House"
4. "Namaste Kitchen"
5. "Flavors of the East"
6. "Masala Magic"
7. "Saffron Bistro"
8. "Chaat Corner"
9. "Tandoori Delight"
10. "Biryani Bliss"


In [17]:
name = chain.run("Italian")
print(name)



"La Bella Cucina" (The Beautiful Kitchen)


In [18]:
print(chain.memory.buffer)

Human: Indian
AI: 

1. "Spice Junction"
2. "Taste of India"
3. "Curry House"
4. "Namaste Kitchen"
5. "Flavors of the East"
6. "Masala Magic"
7. "Saffron Bistro"
8. "Chaat Corner"
9. "Tandoori Delight"
10. "Biryani Bliss"
Human: Italian
AI: 

"La Bella Cucina" (The Beautiful Kitchen)


In [20]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [21]:
convo.run("where did the 2011 world cup held?")

' The 2011 World Cup was held in India, Sri Lanka, and Bangladesh. The final match was held at the Wankhede Stadium in Mumbai, India. The tournament consisted of a total of 49 matches, with the first match being held on February 19th, 2011 at the Sher-e-Bangla National Stadium in Dhaka, Bangladesh.'

In [22]:
convo.run("what is 5+5")

'  5+5 equals 10.'

In [23]:
convo.run("who is the captain of runners up team?")

' The captain of the runners up team in the 2011 World Cup was Kumar Sangakkara, who led the Sri Lankan cricket team. They lost to India in the final match.'

In [24]:
print(convo.memory.buffer)

Human: where did the 2011 world cup held?
AI:  The 2011 World Cup was held in India, Sri Lanka, and Bangladesh. The final match was held at the Wankhede Stadium in Mumbai, India. The tournament consisted of a total of 49 matches, with the first match being held on February 19th, 2011 at the Sher-e-Bangla National Stadium in Dhaka, Bangladesh.
Human: what is 5+5
AI:   5+5 equals 10.
Human: who is the captain of runners up team?
AI:  The captain of the runners up team in the 2011 World Cup was Kumar Sangakkara, who led the Sri Lankan cricket team. They lost to India in the final match.


In [25]:
from langchain.memory import ConversationBufferWindowMemory #restricting the memory of the conversation

memory = ConversationBufferWindowMemory(k=1) #k is the number of the conversations that need to memorize

convo = ConversationChain(llm=OpenAI(temperature=0.6),
                         memory=memory
                         )